In [1]:
from autogen_agentchat.agents import CodingAssistantAgent, AssistantAgent
from autogen_agentchat.task import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_ext.models import AzureOpenAIChatCompletionClient

import os
from dotenv import load_dotenv

In [2]:
# Load environment variables from a .env file
load_dotenv()

# Access environment variables
AZURE_OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
AZURE_OPENAI_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')
AZURE_OPENAI_DEPLOYMENT = os.getenv('AZURE_OPENAI_DEPLOYMENT')

In [3]:
client = AzureOpenAIChatCompletionClient(
    model=AZURE_OPENAI_DEPLOYMENT,
    api_version="2024-02-01",
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    azure_key=AZURE_OPENAI_API_KEY,
    model_capabilities={
        "vision":True,
        "function_calling":True,
        "json_output":True,
    }
)

In [4]:
planner_agent = AssistantAgent(
    "planner_agent",
    model_client=client,
    description="A helpful assistant that can plan trips.",
    system_message="You are a helpful assistant that can suggest a travel plan for a user based on their request.",
)

local_agent = AssistantAgent(
    "local_agent",
    model_client=client,
    description="A local assistant that can suggest local activities or places to visit.",
    system_message="You are a helpful assistant that can suggest authentic and interesting local activities or places to visit for a user and can utilize any context information provided.",
)

language_agent = AssistantAgent(
    "language_agent",
    model_client=client,
    description="A helpful assistant that can provide language tips for a given destination.",
    system_message="You are a helpful assistant that can review travel plans, providing feedback on important/critical tips about how best to address language or communication challenges for the given destination. If the plan already includes language tips, you can mention that the plan is satisfactory, with rationale.",
)

travel_summary_agent = AssistantAgent(
    "travel_summary_agent",
    model_client=client,
    description="A helpful assistant that can summarize the travel plan.",
    system_message="You are a helpful assistant that can take in all of the suggestions and advice from the other agents and provide a detailed tfinal travel plan. You must ensure th b at the final plan is integrated and complete. YOUR FINAL RESPONSE MUST BE THE COMPLETE PLAN. When the plan is complete and all perspectives are integrated, you can respond with TERMINATE.",
)

In [5]:
termination = TextMentionTermination("TERMINATE")

group_chat = RoundRobinGroupChat(
    [planner_agent, local_agent, language_agent, travel_summary_agent],
    termination_condition=termination
)

In [6]:
result = await group_chat.run(task="Plan a 3 day trip to Paris.")

In [7]:
from IPython.display import Markdown, display

total_prompt_tokens = 0
total_completion_tokens = 0

for index, message in enumerate(result.messages, start=1):
    display(Markdown('<hr>'))
    text = f"## Step: {index} (output from {message.source})"
    display(Markdown(text))

    if message.models_usage:
        print(f'Prompt Tokens: {message.models_usage.prompt_tokens}')
        print(f'Completion Tokens: {message.models_usage.completion_tokens}')
        total_prompt_tokens += message.models_usage.prompt_tokens
        total_completion_tokens += message.models_usage.completion_tokens
    display(Markdown('<hr>'))

    display(Markdown(message.content))

display(Markdown('<hr>'))
print(f'Total Prompt Tokens: {total_prompt_tokens}')
print(f'Total Completion Tokens: {total_completion_tokens}')
print(f'Total Tokens: {total_prompt_tokens + total_completion_tokens}')

<hr>

## Step: 1 (output from user)

<hr>

Plan a 3 day trip to Paris.

<hr>

...

Prompt Tokens: 1724
Completion Tokens: 860


<hr>

Here's a comprehensive and integrated 3-day trip plan to Paris, combining itinerary, local advice, and language tips:

### Day 1: Iconic Landmarks

**Morning:**
- **Eiffel Tower:** Kickstart your journey by visiting the world-renowned Eiffel Tower. Book your tickets online ahead of time to avoid lengthy queues. Whether you travel to the summit or admire the tower from the ground, it's a must-visit.
- **Trocadéro Gardens:** Take a leisurely stroll to these gardens for some of the best photo opportunities of the Eiffel Tower.

**Afternoon:**
- **Seine River Cruise:** Take a scenic cruise along the Seine River. It's an ideal way to see major landmarks like Notre-Dame Cathedral and the Louvre while enjoying a unique perspective of the city.

**Evening:**
- **Champs-Élysées and Arc de Triomphe:** Wander down this illustrious avenue and indulge in some shopping or simply a cup of coffee. Head to the Arc de Triomphe and, if you're up for it, climb to the top for sweeping views of Paris.
- **Dinner in Montmartre:** End your day with dinner in the artistic neighborhood of Montmartre, perhaps at a cozy bistro. Try French favorites like escargots or coq au vin. Capture the charm of this vibrant area with a visit to the iconic Sacré-Cœur Basilica, enjoying the city's nocturnal views.

### Day 2: Art and Culture

**Morning:**
- **Louvre Museum:** Spend the morning exploring the Louvre Museum, home to world-famous pieces like the Mona Lisa and the Venus de Milo. Pre-book your tickets to maximize your time.

**Afternoon:**
- **Tuileries Garden:** Rest and enjoy a stroll through these scenic gardens.
- **Musée d'Orsay:** A short walk away, visit the Musée d'Orsay. This museum in a former railway station boasts a fantastic collection of Impressionist and Post-Impressionist masterpieces.

**Evening:**
- **Saint-Germain-des-Prés:** Immerse yourself in this historic district known for its vibrant café culture. Dine at a classic French bistro followed by an evening of live jazz, which the area is renowned for.

### Day 3: Royal Tour and Bohemian Vibes

**Morning:**
- **Palace of Versailles:** Venture to the Palace of Versailles via a short train ride. Tour the sumptuous palace, the Hall of Mirrors, and explore its regal gardens. Consider renting a bike or rowboat here for a leisurely afternoon.

**Afternoon:**
- **Le Marais:** Back in Paris, explore Le Marais. Known for its charming streets, boutiques, and art galleries. Visit the Picasso Museum and delight in a famous falafel at L'As du Fallafel.

**Evening:**
- **Latin Quarter:** Spend your last evening in Paris at the lively Latin Quarter. Explore its historic streets, soak in its spirited ambiance, and enjoy dinner at a local eatery. Visit the Panthéon if time allows.

**Optional Nightcap:**
- **Moulin Rouge:** For an iconic Parisian nightlife experience, consider a show at the Moulin Rouge or a similar classic venue.

### Language and Communication Tips:

1. **Basic French Phrases:** Though English is widely spoken, knowing basics like "Bonjour" (Hello), "Merci" (Thank you), "Excusez-moi" (Excuse me), and "L'addition, s'il vous plaît" (The bill, please) can go a long way.
2. **Mobile Translation Apps:** Equip yourself with apps like Google Translate. It can be especially handy at restaurants or in less tourist-heavy locations.
3. **Etiquette:** Always greet with a "Bonjour" when entering stores or restaurants and say "Merci, au revoir" as you leave.

### Travel Tips:
- **Transport:** Make good use of Paris's efficient Métro system with a multi-day pass for convenience.
- **Advance Bookings:** Ensure tickets for busy attractions are booked in advance to save time.
  
Enjoy every moment in the enchanting City of Light, immersing yourself in its beauty, history, and culture. Safe travels!

TERMINATE

<hr>

Total Prompt Tokens: 3938
Total Completion Tokens: 2473
Total Tokens: 6411
